In [1]:
#---------------------------------------------------------------------------------
#   Author Name:        Camm Perera             
#   Create Date:        11-15-2022
#   Description:        DSE-203 - Group #5, NFL-CTE Knowledge Graph
#   System specs:       
#        MacOS Monterey   : 12.5.1 
#        Python           : 3.8.13 
#        IPython          : 8.4.0
#        ipykernel        : 6.15.2
#        ipywidgets       : 7.6.5
#        jupyter_client   : 6.1.12
#        jupyter_core     : 4.10.0
#        jupyter_server   : 1.18.1
#        jupyterlab       : 3.4.4
#        nbclient         : 0.5.13
#        nbconvert        : 6.4.4
#        nbformat         : 5.5.0
#        notebook         : 6.4.12
#        qtconsole        : 5.3.2
#        traitlets        : 5.1.1
# #---------------------------------------------------------------------------------

In [2]:
import sys
import py_stringmatching as sm 
import py_entitymatching as em
import pandas as pd
import numpy as np 
import re, string, math, time
import wikipedia
import stanza
import requests
from bs4 import BeautifulSoup
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

ModuleNotFoundError: No module named 'wikipedia'

In [ ]:
basic_stats_df = em.read_csv_metadata("../../Datasets/Basic_Stats.csv" ,key='Player Id')
# career_stats_defensive_df = pd.read_csv("./Datasets/Career_Stats_Defensive.csv", low_memory=False)
# career_stats_field_goal_kickers_df = pd.read_csv("./Datasets/Career_Stats_Field_Goal_Kickers.csv", low_memory=False)
# career_stats_fumbles_df = pd.read_csv("./Datasets/Career_Stats_Fumbles.csv", low_memory=False)
# career_stats_kick_return_df = pd.read_csv("./Datasets/Career_Stats_Kick_Return.csv", low_memory=False)
# career_stats_kickoff_df = pd.read_csv("./Datasets/Career_Stats_Kickoff.csv", low_memory=False)
# career_stats_offensive_line_df = pd.read_csv("./Datasets/Career_Stats_Offensive_Line.csv", low_memory=False)
# career_stats_passing_df = pd.read_csv("./Datasets/Career_Stats_Passing.csv", low_memory=False)
# career_stats_punt_return_df = pd.read_csv("./Datasets/Career_Stats_Punt_Return.csv", low_memory=False)
# career_stats_punting_df = pd.read_csv("./Datasets/Career_Stats_Punting.csv", low_memory=False)
# career_stats_receiving_df = pd.read_csv("./Datasets/Career_Stats_Receiving.csv", low_memory=False)
# career_stats_rushing_df = pd.read_csv("./Datasets/Career_Stats_Rushing.csv", low_memory=False)


In [3]:
basic_stats_df

NameError: name 'basic_stats_df' is not defined

In [5]:
# len(basic_stats_df), len(career_stats_defensive_df), len(career_stats_field_goal_kickers_df), len(career_stats_fumbles_df) \
# ,len(career_stats_kick_return_df), len(career_stats_kickoff_df), len(career_stats_offensive_line_df) ,len(career_stats_passing_df) \
# ,len(career_stats_punt_return_df) ,len(career_stats_punting_df) ,len(career_stats_receiving_df) , len(career_stats_rushing_df)

### Extract Wikipedia Data

In [6]:
wiki_title = 'List of NFL players with chronic traumatic encephalopathy'
wiki_url = 'https://en.wikipedia.org/wiki/List_of_NFL_players_with_chronic_traumatic_encephalopathy'

# Python Wikipedia library
wiki_page_object     = wikipedia.page(wiki_title)

# Python Beautiful Soup
wiki_page = requests.get(wiki_url)
soup = BeautifulSoup(wiki_page.content, "lxml")

# print(soup.prettify())

#### Stanza - stanford NLP

In [7]:
nlp = stanza.Pipeline('en', processors='tokenize,mwt,ner', use_gpu=False, pos_batch_size=3000, download_method=None)  # This sets up a default neural pipeline in English

2022-12-03 20:15:59 WARNING: Can not find mwt: default from official model list. Ignoring it.
2022-12-03 20:15:59 INFO: Loading these models for language: en (English):
| Processor | Package   |
-------------------------
| tokenize  | combined  |
| ner       | ontonotes |

2022-12-03 20:15:59 INFO: Use device: cpu
2022-12-03 20:15:59 INFO: Loading: tokenize
2022-12-03 20:15:59 INFO: Loading: ner
2022-12-03 20:16:00 INFO: Done loading processors!


#### Process Players Affected Wiki Section

In [8]:
# Lists to store player names by category
players_affected_ls = []

# Wiki-Extract Players affected
players_affected_ls = soup.select('p')[4:8] 

# Set start time to calculate compute time
start_time = time.time()

# Create lists to hold ORG & Person lists
affected_players_ls = []

# PASS-1: Compute NER with wiki page links - Former players with CTE confirmed post-mortem
for index in players_affected_ls:
    doc = nlp(str(index))                                 

    # Extract PERSON & ORG entities
    for ent in doc.ents:
        if (ent.type =='PERSON'):
            clean_name = re.split('</a', ent.text)[0] 
            affected_players_ls.append(clean_name)
            
# Dedupe list contents
affected_players_ls = [*set(affected_players_ls)]

print("Exec time --- %s seconds ---" % (time.time() - start_time)) 
print(f'# of person: {len(affected_players_ls)}')

Exec time --- 8.828011989593506 seconds ---
# of person: 9


#### Process Former Players affected with CTE Wiki Section

In [9]:
# Lists to store player names by category
former_players_post_mortem_ls = []
pm_former_players_ls = []

# Wiki-Extract Former players with CTE confirmed post-mortem
results = soup.select('ul')[1]
former_players_post_mortem_ls = results.find_all("a")

# Set start time to calculate compute time
start_time = time.time()

# PASS-1: Compute NER with wiki page links - Former players with CTE confirmed post-mortem
for index in former_players_post_mortem_ls:
    doc = nlp(str(index))                                 

    # Extract PERSON & ORG entities
    for ent in doc.ents:
        if (ent.type =='PERSON'):
            clean_name = re.split('</a', ent.text)[0] 
            pm_former_players_ls.append(clean_name)
            
# Dedupe list contents
pm_former_players_ls = [*set(pm_former_players_ls)]

print("Exec time --- %s seconds ---" % (time.time() - start_time)) 
print(f'# of person: {len(pm_former_players_ls)}')

Exec time --- 32.69153308868408 seconds ---
# of person: 63


#### Process Deceased players suspected of having had CTE Wiki Sction

In [10]:
# Lists to store player names by category
deceased_players_ls = []
suspected_deceased_players_ls = []

# Wiki-Extract Former players with CTE confirmed post-mortem
results = soup.select('ul')[2]
deceased_players_ls = results.find_all("a")

# Set start time to calculate compute time
start_time = time.time()

# PASS-1: Compute NER with wiki page links - Former players with CTE confirmed post-mortem
for index in deceased_players_ls:
    doc = nlp(str(index))                                 

    # Extract PERSON & ORG entities
    for ent in doc.ents:
        if (ent.type =='PERSON'):
            clean_name = re.split('</a', ent.text)[0] 
            suspected_deceased_players_ls.append(clean_name)
            
# Dedupe list contents
suspected_deceased_players_ls = [*set(suspected_deceased_players_ls)]

print("Exec time --- %s seconds ---" % (time.time() - start_time)) 
print(f'# of person: {len(suspected_deceased_players_ls)}')

Exec time --- 2.46170711517334 seconds ---
# of person: 5


#### Process Living former players diagnosed with CTE or ALS or reporting symptoms consistent with CTE or ALS Wiki Section

In [11]:
# Lists to store player names by category
former_cte_als_players_ls = []
cte_als_former_players_ls = []

# Wiki-Extract Former players with CTE confirmed post-mortem
results = soup.select('ul')[3]
former_cte_als_players_ls = results.find_all("a")

# Set start time to calculate compute time
start_time = time.time()

# PASS-1: Compute NER with wiki page links - Former players with CTE confirmed post-mortem
for index in former_cte_als_players_ls:
    doc = nlp(str(index))                                 

    # Extract PERSON & ORG entities
    for ent in doc.ents:
        if (ent.type =='PERSON'):
            clean_name = re.split('</a', ent.text)[0] 
            cte_als_former_players_ls.append(clean_name)
            
# Dedupe list contents
cte_als_former_players_ls = [*set(cte_als_former_players_ls)]

print("Exec time --- %s seconds ---" % (time.time() - start_time)) 
print(f'# of person: {len(cte_als_former_players_ls)}')

Exec time --- 16.999281883239746 seconds ---
# of person: 34


#### Process Former players listed as plaintiffs in lawsuits against the NFL for concussion-related injuries received after Wiki playing Section

In [12]:
# Lists to store player names by category
players_lawsuits_nfl_ls = []
players_nfl_lawsuits_ls = []

# Wiki-Extract Former players with CTE confirmed post-mortem
results = soup.select('ul')[4]
players_lawsuits_nfl_ls = results.find_all("a")

# Set start time to calculate compute time
# start_time = time.time()

# PASS-1: Compute NER with wiki page links - Former players with CTE confirmed post-mortem

docs_in = [stanza.Document([], text=str(d)) for d in players_lawsuits_nfl_ls]
docs_out = nlp(docs_in)

In [13]:
for doc in docs_out:
    # Extract PERSON & ORG entities
    for ent in doc.ents:
        if (ent.type =='PERSON'):
            clean_name = re.split('</a', ent.text)[0] 
            players_nfl_lawsuits_ls.append(clean_name)
            
# Dedupe list contents
players_nfl_lawsuits_ls = [*set(players_nfl_lawsuits_ls)]

# print("Exec time --- %s seconds ---" % (time.time() - start_time)) 
print(f'# of person: {len(players_nfl_lawsuits_ls)}')

# of person: 1879


### Text Normalization & Preprocessing

In [14]:
# ---------------------------------------------------
# Normalize player "Name" in Kaggle basic stats
# ---------------------------------------------------
basic_stats_df['Clean_Name'] = basic_stats_df.Name.str.lower().map(lambda s: s.split()[1] + ' ' + s.split()[0]).replace('[^\w\s]',' ', regex=True).str.strip()

In [15]:
# ---------------------------------------------------
# Remove punctuations & lower name 
# ---------------------------------------------------
def remove_punc(name):
    punc = '''!()-[]{};:'"\, <>./?@#$%^&*_~'''
    for ele in name:  
        if ele in punc:  
            names = name.replace(ele, " ") 
    return name.lower().strip()

affected_players_ls  = [remove_punc(i) for i in affected_players_ls]
pm_former_players_ls = [remove_punc(i) for i in pm_former_players_ls]
suspected_deceased_players_ls  = [remove_punc(i) for i in suspected_deceased_players_ls]
cte_als_former_players_ls  = [remove_punc(i) for i in cte_als_former_players_ls]
players_nfl_lawsuits_ls  = [remove_punc(i) for i in players_nfl_lawsuits_ls]

In [16]:
# --------------------------------------------------------
# Create combo dataframe for each list(above) category 
# --------------------------------------------------------
affected_players_df = pd.DataFrame(data=[['affected_players'] * len(affected_players_ls), affected_players_ls]).T
affected_players_df.columns = ['cte_category', 'Clean_Name']

pm_former_players_df = pd.DataFrame(data=[['pm_former_players'] * len(pm_former_players_ls), pm_former_players_ls]).T
pm_former_players_df.columns = ['cte_category', 'Clean_Name']

suspected_deceased_players_df = pd.DataFrame(data=[['suspected_deceased_players'] * len(suspected_deceased_players_ls), suspected_deceased_players_ls]).T
suspected_deceased_players_df.columns = ['cte_category', 'Clean_Name']

cte_als_former_players_df = pd.DataFrame(data=[['cte_als_former_players'] * len(cte_als_former_players_ls), cte_als_former_players_ls]).T
cte_als_former_players_df.columns = ['cte_category', 'Clean_Name']

players_nfl_lawsuits_df = pd.DataFrame(data=[['players_nfl_lawsuits'] * len(players_nfl_lawsuits_ls), players_nfl_lawsuits_ls]).T
players_nfl_lawsuits_df.columns = ['cte_category', 'Clean_Name']

# Combine dataframes
frames = [affected_players_df, pm_former_players_df, suspected_deceased_players_df, cte_als_former_players_df, \
                      cte_als_former_players_df, players_nfl_lawsuits_df]
wiki_cte_players_df = pd.concat(frames)
wiki_cte_players_df

,cte_category,Clean_Name
0,affected_players,ray easterling
1,affected_players,john mackey
2,affected_players,bo jackson
3,affected_players,johnson
4,affected_players,stabler
...,...,...
1874,players_nfl_lawsuits,thomas c. randolph
1875,players_nfl_lawsuits,henri crockett
1876,players_nfl_lawsuits,larry johnson
1877,players_nfl_lawsuits,reidel anthony


#### Create CSV file and em.DataFrame for Entity Matching 

In [18]:
# Create CSV & entity match dataframe for blocking
wiki_cte_players_df['rec_id'] = range(1, 1+len(wiki_cte_players_df))
wiki_cte_players_df.to_csv("../../Datasets/wiki_cte_players_df.csv")
wiki_person_df = em.read_csv_metadata("../../Datasets/wiki_cte_players_df.csv", key='rec_id')
wiki_person_df

Metadata file is not present in the given path; proceeding to read the csv file.


,Unnamed: 0,cte_category,Clean_Name,rec_id
0,0,affected_players,ray easterling,1
1,1,affected_players,john mackey,2
2,2,affected_players,bo jackson,3
3,3,affected_players,johnson,4
4,4,affected_players,stabler,5
...,...,...,...,...
2019,1874,players_nfl_lawsuits,thomas c. randolph,2020
2020,1875,players_nfl_lawsuits,henri crockett,2021
2021,1876,players_nfl_lawsuits,larry johnson,2022
2022,1877,players_nfl_lawsuits,reidel anthony,2023


#### Block DataFrames to get Candidate set

In [19]:
#  Instantiate blocker objects:
# ------------------------------
# Create overlap blocker
ob = em.OverlapBlocker()

# Create attribute equivalence blocker
ab = em.AttrEquivalenceBlocker()

#### i. Overlap Block by 'player_name'

In [20]:
# # Block tables using full name
ob_fullname_cand = ob.block_tables(basic_stats_df, wiki_person_df, 'Clean_Name', 'Clean_Name', allow_missing=False,
                                l_output_attrs=['Player Id', 'Name',  'Age', 'Current Status', 'Birthday', 'College','High School', 'Clean_Name'],
                                r_output_attrs=['rec_id', 'Clean_Name', 'cte_category'],
                                overlap_size=2, verbose=False)

/Users/joel/opt/anaconda3/lib/python3.9/site-packages/py_entitymatching/blocker/overlap_blocker.py:258: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  l_df[l_dummy_overlap_attr] = l_df[l_overlap_attr]
/Users/joel/opt/anaconda3/lib/python3.9/site-packages/py_entitymatching/blocker/overlap_blocker.py:615: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  table[overlap_attr] = values
/Users/joel/opt/anaconda3/lib/python3.9/site-packages/py_stringsimjoin/utils/generic_helper.py:76: FutureWarning: In a future version

In [22]:
#  Matched candidates - Kaggle vs. Wiki page
ob_fullname_cand.to_csv('../../Datasets/OB_names_matched.csv')
ob_fullname_cand

,_id,ltable_Player Id,rtable_rec_id,ltable_Name,ltable_Age,ltable_Current Status,ltable_Birthday,ltable_College,ltable_High School,ltable_Clean_Name,rtable_Clean_Name,rtable_cte_category
0,0,rayeasterling/2513421,1,"Easterling, Ray",NaN,Retired,9/3/1949,Richmond,NaN,ray easterling,ray easterling,affected_players
1,1,johnmackey/2519920,2,"Mackey, John",NaN,Retired,9/24/1941,Syracuse,NaN,john mackey,john mackey,affected_players
2,2,bojackson/2517329,3,"Jackson, Bo",54.0,Retired,11/30/1962,Auburn,NaN,bo jackson,bo jackson,affected_players
3,3,larryjohnson/2505491,7,"Johnson, Larry",37.0,Retired,11/19/1979,Penn State,NaN,larry johnson,larry johnson,affected_players
4,4,larryjohnson/2517701,7,"Johnson, Larry",108.0,Retired,3/28/1909,Haskell Indian,NaN,larry johnson,larry johnson,affected_players
...,...,...,...,...,...,...,...,...,...,...,...,...
878,878,thomasrandolph/2502582,2020,"Randolph, Thomas",46.0,Retired,10/5/1970,Kansas State,NaN,thomas randolph,thomas c. randolph,players_nfl_lawsuits
879,879,larryjohnson/2505491,2022,"Johnson, Larry",37.0,Retired,11/19/1979,Penn State,NaN,larry johnson,larry johnson,players_nfl_lawsuits
880,880,larryjohnson/2517701,2022,"Johnson, Larry",108.0,Retired,3/28/1909,Haskell Indian,NaN,larry johnson,larry johnson,players_nfl_lawsuits
881,881,reidelanthony/2499442,2023,"Anthony, Reidel",40.0,Retired,10/20/1976,Florida,NaN,reidel anthony,reidel anthony,players_nfl_lawsuits


In [23]:
# # Debug blocker output : (FOR TESTING ONLY)
# #  Unmatched candidates - Kaggle vs. Wiki page
# corres = [('Clean_Name', 'Clean_Name')]
# ob_fullname_debug = em.debug_blocker(ob_fullname_cand, basic_stats_df, wiki_person_df, output_size=500, attr_corres=corres)

# # Display first few tuple pairs from the debug_blocker's output
# ob_fullname_debug  #.to_csv('./names_debug.csv')

#### ii. Attribute Block by 'player_name'

### <font color='red'> RESULTS BETTER THAN OVERLAP BLOCK</font>

In [24]:
# Block using 'full_name_dob' attribute
ab_fullname_cand = ab.block_tables(basic_stats_df, wiki_person_df, 'Clean_Name', 'Clean_Name', allow_missing=False,
                                l_output_attrs=['Player Id', 'Name',  'Age', 'Current Status', 'Birthday', 'College','High School', 'Clean_Name'],
                                r_output_attrs=['rec_id', 'Clean_Name', 'cte_category'], n_jobs=2)

In [25]:
#  Distinct matched candidates - Kaggle vs. Wiki page
ab_fullname_cand.groupby("ltable_Player Id").first().to_csv('./AB_names_matched.csv')
ab_fullname_cand.groupby("ltable_Player Id").first()

,_id,rtable_rec_id,ltable_Name,ltable_Age,ltable_Current Status,ltable_Birthday,ltable_College,ltable_High School,ltable_Clean_Name,rtable_Clean_Name,rtable_cte_category
ltable_Player Id,,,,,,,,,,,
aaronbeasley/2499587,336,598,"Beasley, Aaron",43.0,Retired,7/7/1973,West Virginia,None,aaron beasley,aaron beasley,players_nfl_lawsuits
aaronjones/2558116,391,1367,"Jones, Aaron",22.0,Active,12/2/1994,Texas-El Paso,Burges HS,aaron jones,aaron jones,players_nfl_lawsuits
adamhaayer/2504632,348,447,"Haayer, Adam",40.0,Retired,2/22/1977,Minnesota,None,adam haayer,adam haayer,players_nfl_lawsuits
adriandingle/2500398,122,445,"Dingle, Adrian",39.0,Retired,6/25/1977,Clemson,None,adrian dingle,adrian dingle,players_nfl_lawsuits
adrianyoung/2529565,258,489,"Young, Adrian",71.0,Retired,1/31/1946,USC,None,adrian young,adrian young,players_nfl_lawsuits
...,...,...,...,...,...,...,...,...,...,...,...
willieoshodin/2502367,475,1750,"Oshodin, Willie",47.0,Retired,9/16/1969,Villanova,None,willie oshodin,willie oshodin,players_nfl_lawsuits
willietaylor/2527127,726,1435,"Taylor, Willie",61.0,Retired,12/9/1955,Pittsburgh,None,willie taylor,willie taylor,players_nfl_lawsuits
woodythompson/2527305,308,183,"Thompson, Woody",64.0,Retired,8/20/1952,Miami (Fla.),None,woody thompson,woody thompson,players_nfl_lawsuits
